In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.signal import find_peaks, peak_widths
import matplotlib.pyplot as plt

def calculate_metrics(data, sigma_multiplier=4):
    mean = data.values.mean()
    sigma = np.std(data.values)

    # STD
    std_deviation = sigma

    # Anomaly Count
    upper_limit = mean + sigma_multiplier * sigma
    lower_limit = mean - sigma_multiplier * sigma
    anomaly_count = ((data.values < lower_limit) | (data.values > upper_limit)).sum()

    # Widest Peak Width
    flattened_data = data.values.flatten()
    peaks, _ = find_peaks(flattened_data)
    widths = peak_widths(flattened_data, peaks, rel_height=0.5)[0]
    widest_peak_width = max(widths) if widths.size > 0 else 0

    return std_deviation, anomaly_count, widest_peak_width

def anomaly_scoring(data, alpha=1.0, beta=0.5, gamma=1.0):
    x1, x2, x3 = calculate_metrics(data)
    
    # 가중합 계산
    weighted_score = (alpha * x1) + (beta * x2) + (gamma * x3)
    
    return weighted_score

In [ ]:
# INPUT
path = '../csv/battery_1st/ng02_ng01.csv'

data = pd.read_csv(path, names=["value"])
score = anomaly_scoring(data)

# OUTPUT
print(f"Anomaly Score: {score}")

# 참고용 - 1차 데이터

In [ ]:
dir_path = '../csv/battery_1st'

for file in sorted(os.listdir(dir_path)):
    data = pd.read_csv(os.path.join(dir_path,file), names=["value"])
    score = anomaly_scoring(data)

    print(f"Anomaly Score of {file}: {score}")

In [ ]:
dir_path = '../csv/battery_1st'

for file in sorted(os.listdir(dir_path)):
    data = pd.read_csv(os.path.join(dir_path,file), names=["value"])
    std_deviation, anomaly_count, widest_peak_width = calculate_metrics(data)

    print(f"Metrics of {file}: STD {std_deviation}/ AC {anomaly_count}/ WPW {widest_peak_width}")